In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import numpy as np
import random
import pickle 

def scrape_goodreads_ratings(user_id, max_pages=10):
    """
    Scrape a user's star ratings from Goodreads.
    
    Args:
    - user_id (str): Goodreads user ID or profile suffix.
    - max_pages (int): Maximum number of pages to scrape (each page contains ~30 books).
    
    Returns:
    - pd.DataFrame: A DataFrame containing book titles and ratings.
    """
    base_url = f"https://www.goodreads.com/review/list/{user_id}?shelf=read"
    headers = {"User-Agent": "Mozilla/5.0"}
    books = []

    for page in range(1, max_pages + 1):
        url = f"{base_url}&page={page}"
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find all book entries in the table
        rows = soup.find_all("tr", class_="bookalike review")
        if not rows:
            print("No more data found.")
            break

        for row in rows:
            try:
                title = row.find("td", class_="field title").a.text.strip()
                rating_element = row.find("td", class_="field rating")
                rating = rating_element.find("span", class_="staticStars").get("title", "No rating")
                stars = map_rating(rating)
                books.append({"Title": title, "Rating": stars, "User_id": user_id})
#                 print(title, rating, stars)
            except AttributeError:
                # Handle rows with missing data
                continue

        print(f"Page {page} scraped successfully.")
        time.sleep(random.uniform(1, 5))  # Be kind to the server and avoid being blocked

    # Return data as a pandas DataFrame
    return pd.DataFrame(books)



In [2]:
def map_rating(phrase):
    rating_map = {
        "liked it": 3,
        "really liked it": 4,
        "it was ok": 2, 
        "it was amazing": 5, 
        "did not like it": 1,
    }
    
    return rating_map.get(phrase, "Invalid rating")  # Default to "Invalid rating" if the phrase isn't in the dictionary


In [3]:
# if __name__ == "__main__":
# #     user_id = "6688207"  # Replace with the Goodreads user ID or profile suffix
# #     for user_id in tqdm(['30181442', '75009563', '11345366', '110912303', '113964939', '11215896', '53701594', '4622890', '93628736', '176180116']):
# #     for user_id in tqdm(['2974095', '4622890', '28953843', '16174645', '4159922', '4125660', '54886546', '16912659', '260116', '4685500', '21865425']):
# #     for user_id in tqdm(['53701594', '27709782', '7566229', '16652861', '30817744', '56259255', '4125660', '60964126', 
# #                          '176167767', '28510930', '1029975', '131020767', '28862120', '88713906', '160141433', '41097916', 
# #                          '20809863', '69519261', '24017481', '7376365', '75941333', '13571407', '106618742', '17792052',
# #                          '3534528', '130656897', '7474475', '4125412', '6336365', '6026811', '3438047']):
#     for user_id in ['169695556']:
#         print("User_id = ", user_id)
#         max_pages = 30  # Adjust based on expected data
#         ratings_data = scrape_goodreads_ratings(user_id, max_pages)

#         if not ratings_data.empty:
# #             print(ratings_data.head())
# #             ratings_data.to_csv("goodreads_ratings.csv", index=False)
#             ratings_data.to_csv('goodreads_ratings.csv', mode='a', header=False, index=False)
#             print("Data saved to goodreads_ratings.csv.")
#         else:
#             print("No data retrieved.")


In [4]:
df = pd.read_csv('goodreads_ratings_series.csv')
print(df.shape)
df = df.drop_duplicates()
# Print the entire DataFrame
print(df)

duplicate_count = df['Title'].duplicated().sum()
print("Number of books with at least two people rating it:", duplicate_count)
duplicate_counts_per_value = df['Title'].value_counts()
print(duplicate_counts_per_value)
print("Number of unique books: ", df['Title'].nunique())
num_users = df['User_id'].nunique()
user_ids = list(df['User_id'].unique())
print("number of users is: ", num_users)
print("user_ids = ", user_ids)

/tmp/ipykernel_2614/1922004947.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('goodreads_ratings_series.csv')


(253718, 6)
                                                    Title Rating    User_id  \
0                                       I Am Watching You      3  169695558   
1       Three to Get Deadly\n        (Stephanie Plum, #3)      3  169695558   
2       Before the Coffee Gets Cold\n        (Before t...      4  169695558   
3       Dark Sacred Night\n        (Renée Ballard, #2;...      4  169695558   
4         Two for the Dough\n        (Stephanie Plum, #2)      4  169695558   
...                                                   ...    ...        ...   
253713                                          Jane Eyre      1  150076381   
253714   Fifty Shades of Grey\n        (Fifty Shades, #1)      1  150076381   
253715  Harry Potter and the Sorcerer's Stone\n       ...      5  150076381   
253716                                Pride and Prejudice      5  150076381   
253717                 Divergent\n        (Divergent, #1)      4  150076381   

       Series  First  Suggest  
0      

In [5]:
# # Get a list of top titles in order
# top_titles = duplicate_counts_per_value.index.tolist()
# top_100 = top_titles[:100]

# for title in top_100:
#     print(title)
    
# with open("top_100.pkl", "wb") as file:
#     pickle.dump(top_100, file)

In [6]:
# threshold = 5#num_users * 0.1
# pop_titles = list(duplicate_counts_per_value[duplicate_counts_per_value > threshold].index)
# my_titles = df.loc[df["User_id"] == 169695558, "Title"].tolist()
# # print(my_titles)

# print("pop titles len = ", len(pop_titles))
# print(pop_titles)
# print("my titles len = ", len(my_titles))
# titles = list(set(pop_titles))# + my_titles))

# # #remove Harry Potter titles:
# # titles = [s for s in titles if "Harry Potter" not in s]

# num_titles = len(titles)


# print(titles)
# # print(titles)
# print("num_titles =", num_titles)

# # ratings = np.full((num_users, num_titles), None)
# ratings = np.zeros((num_users, num_titles))

# for index, row in df.iterrows():
#     if row['Title'] in titles:
#         try:
#             ratings[user_ids.index(row['User_id']), titles.index(row["Title"])] = int(row["Rating"])
# #             print("found ", row["Title"])
#         except:
#             pass
        
# print("ratings size = ", ratings.shape)
# ratings = ratings[~np.all(ratings == 0, axis=1)]
# print("ratings size = ", ratings.shape)
# # Save the list to a file
# with open("titles.pkl", "wb") as file:
#     pickle.dump(titles, file)

In [7]:
# # print(df['Suggest'])

# suggest = list(df['Suggest'])
# print(suggest)

# with open("suggest.pkl", "wb") as file:
#     suggest.dump(suggest, file)

In [8]:
# ratings_df = pd.DataFrame(ratings)
# print(ratings_df.shape)
# #delete users that don't have any of these ratings
# # ratings_df = ratings_df.loc[~(ratings_df == 0).all(axis=1)]
# ratings_df = ratings_df.loc[(ratings_df != 0).sum(axis=1) >= 4] #need at least 4 entries to stay
# print(ratings_df.shape)

# # Calculate percentage of non-zero elements
# percentage_nonzero = (np.count_nonzero(ratings_df) / ratings_df.size) * 100
# print("percentage_nonzero =", round(percentage_nonzero, 1), '%')

# # Save the list to a file
# with open("ratings_df.pkl", "wb") as file:
#     pickle.dump(ratings_df, file)


In [9]:
with open("titles.pkl", "rb") as file:
    titles = pickle.load(file)
    
with open("top_100.pkl", "rb") as file:
    top_100 = pickle.load(file)
    
with open("suggest.pkl", "rb") as file:
    suggest = pickle.load(file)
    
num_titles = len(titles)

# Load the list from the file
with open("ratings_df.pkl", "rb") as file:
    ratings_df = pickle.load(file)
    ratings = ratings_df.to_numpy()


In [10]:
# Convert the filled ratings data into a numpy array
ratings_matrix = ratings_df.values

# Initialize KNN (using user-based KNN)
import math
knn = NearestNeighbors(n_neighbors=50, metric='cosine')  # Using cosine similarity  math.ceil(num_users/10)
knn.fit(ratings_matrix)

with open("knn_model.pkl", "wb") as file:
    pickle.dump(knn, file)

# Example: Predict rating for User 0 on Item 2
user_id = 0  # Index of user in the matrix
item_id = 2  # Index of item to predict rating for

# Get the nearest neighbors for user 0 (excluding the user itself)
distances, indices = knn.kneighbors([ratings_matrix[user_id]])

pred_ratings_list = np.array([])
rankings_list = np.array([])
for item_id in range(num_titles):
    # Get the ratings for the neighbors on item 2
    neighbor_ratings = np.array([ratings_matrix[i, item_id] for i in indices[0] if not np.isnan(ratings_matrix[i, item_id])])

    predicted_rating = np.mean(neighbor_ratings[np.nonzero(neighbor_ratings)])
    rankings = np.sum(neighbor_ratings[np.nonzero(neighbor_ratings)])

#     print(predicted_rating)

    pred_ratings_list = np.append(pred_ratings_list, predicted_rating)
    rankings_list = np.append(rankings_list, rankings)
#     print(f"Predicted rating for User {user_id} on Item {item_id}: {predicted_rating}")
    
best_book_rating = np.max(pred_ratings_list)
best_book_idx = np.argmax(pred_ratings_list)

sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
n = 1
for i, idx in enumerate(sorted_indices): 
    if (ratings[0, idx] > 0) or (np.isnan(pred_ratings_list[idx])):
        pass
    else:
        print("#", (n) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
        n+=1

/root/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/root/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Top books are:
# 1 Lessons in Chemistry Rating: 5.0
# 2 Dracula Rating: 5.0
# 3 The Merchant of Death
        (Pendragon, #1) Rating: 5.0
# 4 Temple of the Winds
        (Sword of Truth, #4) Rating: 5.0
# 5 The Long Winter
        (Little House, #6) Rating: 5.0
# 6 The Amulet of Samarkand
        (Bartimaeus, #1) Rating: 5.0
# 7 Homegoing Rating: 5.0
# 8 This Lullaby Rating: 5.0
# 9 Anne's House of Dreams
        (Anne of Green Gables, #5) Rating: 5.0
# 10 Aliceâ€™s Adventures in Wonderland / Through the Looking-Glass Rating: 5.0
# 11 Me Talk Pretty One Day Rating: 5.0
# 12 One Last Stop Rating: 5.0
# 13 The Mists of Avalon
        (Avalon, #1) Rating: 5.0
# 14 Lock and Key Rating: 5.0
# 15 Howlâ€™s Moving Castle
        (Howlâ€™s Moving Castle, #1) Rating: 5.0
# 16 Rosencrantz and Guildenstern Are Dead Rating: 5.0
# 17 Siege and Storm
        (The Shadow and Bone Trilogy, #2) Rating: 5.0
# 18 Love on the Brain Rating: 5.0
# 19 Shadow of the Hegemon
        (The Shadow Series, #2) Rati

        (Caraval, #2) Rating: 4.0
# 430 The Tale of Peter Rabbit
        (World of Beatrix Potter, #1) Rating: 4.0
# 431 The Island of Sea Women Rating: 4.0
# 432 The Bean Trees
        (Greer Family, #1) Rating: 4.0
# 433 Marley and Me: Life and Love With the World’s Worst Dog Rating: 4.0
# 434 Fear Nothing
        (Moonlight Bay, #1) Rating: 4.0
# 435 All Creatures Great and Small
        (All Creatures Great and Small, #1-2) Rating: 4.0
# 436 Stone Soup Rating: 4.0
# 437 City of Bones
        (The Mortal Instruments, #1) Rating: 4.0
# 438 Blue Bloods
        (Blue Bloods, #1) Rating: 4.0
# 439 Twenties Girl Rating: 4.0
# 440 Heartstopper: Volume Two
        (Heartstopper, #2) Rating: 4.0
# 441 If You Ask Me Rating: 4.0
# 442 A Midsummer Night’s Dream Rating: 4.0
# 443 Betrayed
        (House of Night, #2) Rating: 4.0
# 444 The First Fifteen Lives of Harry August Rating: 4.0
# 445 A Wrinkle in Time
        (Time Quintet, #1) Rating: 4.0
# 446 A Darker Shade of Magic
        (Shades o

# 781 Stardust Rating: 3.0
# 782 The Gift of the Magi Rating: 3.0
# 783 It's Kind of a Funny Story Rating: 3.0
# 784 Normal People Rating: 3.0
# 785 Dark Places Rating: 3.0
# 786 Perfume: The Story of a Murderer Rating: 3.0
# 787 Charlie and the Great Glass Elevator
        (Charlie Bucket, #2) Rating: 3.0
# 788 Where the Heart Is Rating: 3.0
# 789 Queen of Shadows
        (Throne of Glass, #4) Rating: 3.0
# 790 The Devil Wears Prada
        (The Devil Wears Prada, #1) Rating: 3.0
# 791 Wither
        (The Chemical Garden, #1) Rating: 3.0
# 792 Reached
        (Matched, #3) Rating: 3.0
# 793 Pet Sematary Rating: 3.0
# 794 The Story of Babar
        (Babar, #1) Rating: 3.0
# 795 Heidi Rating: 3.0
# 796 The Rescue Rating: 3.0
# 797 Old Yeller Rating: 3.0
# 798 Lean In: Women, Work, and the Will to Lead Rating: 3.0
# 799 The Mayor of Casterbridge Rating: 3.0
# 800 The Phantom Tollbooth Rating: 3.0
# 801 From Potter's Field
        (Kay Scarpetta, #6) Rating: 3.0
# 802 Tess of the Dâ€™Urbe

In [11]:
#make knn for the similar user part
# Initialize KNN (using user-based KNN)
# import math
knn_30 = NearestNeighbors(n_neighbors=50, metric='cosine')  # Using cosine similarity
knn_30.fit(ratings_matrix)

with open("knn_model_30.pkl", "wb") as file:
    pickle.dump(knn_30, file)

In [12]:
rankings_list

best_book_rating = np.max(rankings_list)
best_book_idx = np.argmax(rankings_list)

sorted_indices = np.argsort(rankings_list)[::-1]
print("Top books are:")
for i, idx in enumerate(sorted_indices): 
    print("#", (i+1) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1), ". Ranking:", rankings_list[idx])

Top books are:
# 1 Harry Potter and the Deathly Hallows
        (Harry Potter, #7) Rating: 4.8 . Ranking: 221.0
# 2 Harry Potter and the Prisoner of Azkaban
        (Harry Potter, #3) Rating: 4.8 . Ranking: 220.0
# 3 Harry Potter and the Goblet of Fire
        (Harry Potter, #4) Rating: 4.8 . Ranking: 218.0
# 4 Harry Potter and the Half-Blood Prince
        (Harry Potter, #6) Rating: 4.7 . Ranking: 211.0
# 5 Harry Potter and the Chamber of Secrets
        (Harry Potter, #2) Rating: 4.6 . Ranking: 208.0
# 6 Harry Potter and the Order of the Phoenix
        (Harry Potter, #5) Rating: 4.7 . Ranking: 206.0
# 7 Harry Potter and the Sorcerer's Stone
        (Harry Potter, #1) Rating: 4.7 . Ranking: 201.0
# 8 The Hunger Games
        (The Hunger Games, #1) Rating: 4.7 . Ranking: 174.0
# 9 Catching Fire
        (The Hunger Games, #2) Rating: 4.6 . Ranking: 142.0
# 10 Mockingjay
        (The Hunger Games, #3) Rating: 4.4 . Ranking: 135.0
# 11 Twilight
        (The Twilight Saga, #1) Rating: 3.9

# 288 Speak Rating: 4.0 . Ranking: 8.0
# 289 Fight Club Rating: 2.7 . Ranking: 8.0
# 290 The Screwtape Letters Rating: 4.0 . Ranking: 8.0
# 291 The Witches Rating: 4.0 . Ranking: 8.0
# 292 Prince Caspian
        (Chronicles of Narnia, #2) Rating: 4.0 . Ranking: 8.0
# 293 Eat, Pray, Love Rating: 2.7 . Ranking: 8.0
# 294 A Good Girl's Guide to Murder
        (A Good Girl's Guide to Murder, #1) Rating: 4.0 . Ranking: 8.0
# 295 The Pact Rating: 4.0 . Ranking: 8.0
# 296 How the Grinch Stole Christmas! Rating: 4.0 . Ranking: 8.0
# 297 The Hiding Place: The Triumphant True Story of Corrie Ten Boom Rating: 4.0 . Ranking: 8.0
# 298 Oh, the Places Youâ€™ll Go! Rating: 4.0 . Ranking: 8.0
# 299 Island of the Blue Dolphins Rating: 2.7 . Ranking: 8.0
# 300 They Both Die at the End
        (Death-Cast, #1) Rating: 4.0 . Ranking: 8.0
# 301 Dead in the Family
        (Sookie Stackhouse, #10) Rating: 4.0 . Ranking: 8.0
# 302 Stargirl
        (Stargirl, #1) Rating: 4.0 . Ranking: 8.0
# 303 Plum Lovin'
  

# 558 Anne of the Island
        (Anne of Green Gables, #3) Rating: 5.0 . Ranking: 5.0
# 559 An Abundance of Katherines Rating: 2.5 . Ranking: 5.0
# 560 Blood of the Fold
        (Sword of Truth, #3) Rating: 5.0 . Ranking: 5.0
# 561 Unearthly
        (Unearthly, #1) Rating: 5.0 . Ranking: 5.0
# 562 One Last Stop Rating: 5.0 . Ranking: 5.0
# 563 Watership Down
        (Watership Down, #1) Rating: 4.0 . Ranking: 4.0
# 564 The House at Pooh Corner
        (Winnie-the-Pooh, #2) Rating: 4.0 . Ranking: 4.0
# 565 The Dinner Rating: 4.0 . Ranking: 4.0
# 566 Plain Truth Rating: 4.0 . Ranking: 4.0
# 567 Alice in Wonderland Rating: 4.0 . Ranking: 4.0
# 568 Running with Scissors Rating: 2.0 . Ranking: 4.0
# 569 A Wind in the Door
        (Time Quintet, #2) Rating: 4.0 . Ranking: 4.0
# 570 There Was an Old Lady Who Swallowed a Fly Rating: 4.0 . Ranking: 4.0
# 571 Circus of the Damned
        (Anita Blake, Vampire Hunter, #3) Rating: 4.0 . Ranking: 4.0
# 572 The Lord of the Rings Rating: 4.0 . Ranki

# 836 The Gift of the Magi Rating: 3.0 . Ranking: 3.0
# 837 Ghosts Rating: 3.0 . Ranking: 3.0
# 838 2nd Chance
        (Women's Murder Club, #2) Rating: 3.0 . Ranking: 3.0
# 839 From the Mixed-Up Files of Mrs. Basil E. Frankweiler Rating: 3.0 . Ranking: 3.0
# 840 The Girls Rating: 3.0 . Ranking: 3.0
# 841 The Friday Night Knitting Club
        (Friday Night Knitting Club, #1) Rating: 3.0 . Ranking: 3.0
# 842 House Rules Rating: 3.0 . Ranking: 3.0
# 843 The Awakening Rating: 3.0 . Ranking: 3.0
# 844 The Phantom Tollbooth Rating: 3.0 . Ranking: 3.0
# 845 The Guest List Rating: 3.0 . Ranking: 3.0
# 846 The Magician’s Nephew
        (Chronicles of Narnia, #6) Rating: 3.0 . Ranking: 3.0
# 847 I Am Watching You Rating: 3.0 . Ranking: 3.0
# 848 Where the Heart Is Rating: 3.0 . Ranking: 3.0
# 849 How the King of Elfhame Learned to Hate Stories
        (The Folk of the Air, #3.5) Rating: 3.0 . Ranking: 3.0
# 850 A Long Way Gone: Memoirs of a Boy Soldier Rating: 3.0 . Ranking: 3.0
# 851 Mary Jan

# 1106 The Other Einstein Rating: nan . Ranking: 0.0
# 1107 The Princess Diarist Rating: nan . Ranking: 0.0
# 1108 Calypso Rating: nan . Ranking: 0.0
# 1109 The Child
        (Kate Waters, #2) Rating: nan . Ranking: 0.0
# 1110 The Bookish Life of Nina Hill Rating: nan . Ranking: 0.0
# 1111 Falling Up Rating: nan . Ranking: 0.0
# 1112 Thinner Rating: nan . Ranking: 0.0
# 1113 Summer of '69 Rating: nan . Ranking: 0.0
# 1114 The Hating Game Rating: nan . Ranking: 0.0
# 1115 The Fine Print
        (Dreamland Billionaires, #1) Rating: nan . Ranking: 0.0
# 1116 Modern Romance Rating: nan . Ranking: 0.0
# 1117 Jaws
        (Jaws, #1) Rating: nan . Ranking: 0.0
# 1118 Even Cowgirls Get the Blues Rating: nan . Ranking: 0.0
# 1119 The Other People Rating: nan . Ranking: 0.0
# 1120 Darkfever
        (Fever, #1) Rating: nan . Ranking: 0.0
# 1121 Talking to Strangers: What We Should Know About the People We Don't Know Rating: nan . Ranking: 0.0
# 1122 Weyward Rating: nan . Ranking: 0.0
# 1123 This 

...

# 1858 Hidden Pictures Rating: nan . Ranking: 0.0
# 1859 White Noise Rating: nan . Ranking: 0.0
# 1860 The Kind Worth Killing
        (Henry Kimball/Lily Kintner, #1) Rating: nan . Ranking: 0.0
# 1861 If Something Happens to Me Rating: nan . Ranking: 0.0
# 1862 Fantastic Beasts and Where to Find Them Rating: nan . Ranking: 0.0
# 1863 A Tale for the Time Being Rating: nan . Ranking: 0.0
# 1864 The Alloy of Law
        (Mistborn, #4) Rating: nan . Ranking: 0.0
# 1865 Pyramids
        (Discworld, #7) Rating: nan . Ranking: 0.0
# 1866 How to Sell a Haunted House Rating: nan . Ranking: 0.0
# 1867 The Man Who Died Twice
        (Thursday Murder Club, #2) Rating: nan . Ranking: 0.0
# 1868 Cannery Row
        (Cannery Row, #1) Rating: nan . Ranking: 0.0
# 1869 Being Mortal: Medicine and What Matters in the End Rating: nan . Ranking: 0.0
# 1870 A Moveable Feast Rating: nan . Ranking: 0.0
# 1871 Fables, Vol. 1: Legends in Exile Rating: nan . Ranking: 0.0
# 1872 Three Men in a Boat
        (Three

# 2123 Half Broke Horses Rating: nan . Ranking: 0.0
# 2124 Hard-Boiled Wonderland and the End of the World Rating: nan . Ranking: 0.0
# 2125 Open House Rating: nan . Ranking: 0.0
# 2126 The Secret Place
        (Dublin Murder Squad, #5) Rating: nan . Ranking: 0.0
# 2127 The Life and Times of the Thunderbolt Kid Rating: nan . Ranking: 0.0
# 2128 Revelations
        (Blue Bloods, #3) Rating: nan . Ranking: 0.0
# 2129 The Lies I Tell Rating: nan . Ranking: 0.0
# 2130 The Thin Man Rating: nan . Ranking: 0.0
# 2131 Preludes & Nocturnes
        (The Sandman, #1) Rating: nan . Ranking: 0.0
# 2132 Baby Proof Rating: nan . Ranking: 0.0
# 2133 The Summer I Turned Pretty
        (Summer, #1) Rating: nan . Ranking: 0.0
# 2134 Hiroshima Rating: nan . Ranking: 0.0
# 2135 The Woods Rating: nan . Ranking: 0.0
# 2136 Howl and Other Poems Rating: nan . Ranking: 0.0
# 2137 The Younger Wife Rating: nan . Ranking: 0.0
# 2138 Moon Called
        (Mercy Thompson, #1) Rating: nan . Ranking: 0.0
# 2139 Interpr

        (After, #1) Rating: nan . Ranking: 0.0
# 2401 The Color of Magic
        (Discworld, #1; Rincewind, #1) Rating: nan . Ranking: 0.0
# 2402 House of Earth and Blood
        (Crescent City, #1) Rating: nan . Ranking: 0.0
# 2403 The Invisible Man Rating: nan . Ranking: 0.0
# 2404 Dear Edward Rating: nan . Ranking: 0.0
# 2405 Beyond the Shadows
        (Night Angel, #3) Rating: nan . Ranking: 0.0
# 2406 61 Hours
        (Jack Reacher, #14) Rating: nan . Ranking: 0.0
# 2407 City of Thieves Rating: nan . Ranking: 0.0
# 2408 Gulp: Adventures on the Alimentary Canal Rating: nan . Ranking: 0.0
# 2409 Flowers in the Attic
        (Dollanganger, #1) Rating: nan . Ranking: 0.0
# 2410 The It Girl Rating: nan . Ranking: 0.0
# 2411 Dial A for Aunties
        (Aunties, #1) Rating: nan . Ranking: 0.0
# 2412 The Last Time I Lied Rating: nan . Ranking: 0.0
# 2413 Crossroads of Twilight
        (The Wheel of Time, #10) Rating: nan . Ranking: 0.0
# 2414 The Silver Chair
        (Chronicles of Narnia

# 2664 Night Road Rating: nan . Ranking: 0.0
# 2665 Love and Other Words Rating: nan . Ranking: 0.0
# 2666 We Are All Completely Beside Ourselves Rating: nan . Ranking: 0.0
# 2667 The Sign of Four
        (Sherlock Holmes, #2) Rating: nan . Ranking: 0.0
# 2668 A Kiss of Shadows
        (Merry Gentry, #1) Rating: nan . Ranking: 0.0
# 2669 The â€‹Crown of Gilded Bones
        (Blood and Ash, #3) Rating: nan . Ranking: 0.0
# 2670 Hogfather
        (Discworld, #20; Death, #4) Rating: nan . Ranking: 0.0
# 2671 Mayflower: A Story of Courage, Community, and War Rating: nan . Ranking: 0.0
# 2672 The Omnivore's Dilemma: A Natural History of Four Meals Rating: nan . Ranking: 0.0
# 2673 The Lincoln Highway Rating: nan . Ranking: 0.0
# 2674 Hello Beautiful Rating: nan . Ranking: 0.0
# 2675 The Prince and the Pauper Rating: nan . Ranking: 0.0
# 2676 Fool Me Once Rating: nan . Ranking: 0.0
# 2677 Vicious
        (Villains, #1) Rating: nan . Ranking: 0.0
# 2678 Memory Man
        (Amos Decker, #1) Ra

        (Night Angel, #1) Rating: nan . Ranking: 0.0
# 2935 Ruthless Vows
        (Letters of Enchantment, #2) Rating: nan . Ranking: 0.0
# 2936 Small Favor
        (The Dresden Files, #10) Rating: nan . Ranking: 0.0
# 2937 My Grandmother Asked Me to Tell You She's Sorry Rating: nan . Ranking: 0.0
# 2938 Empire Falls Rating: nan . Ranking: 0.0
# 2939 The Lunatic Cafe
        (Anita Blake, Vampire Hunter, #4) Rating: nan . Ranking: 0.0
# 2940 In Defense of Food: An Eater's Manifesto Rating: nan . Ranking: 0.0
# 2941 Hyperion
        (Hyperion Cantos, #1) Rating: nan . Ranking: 0.0
# 2942 The Dead Romantics Rating: nan . Ranking: 0.0
# 2943 I Found You Rating: nan . Ranking: 0.0
# 2944 Lincoln in the Bardo Rating: nan . Ranking: 0.0
# 2945 Shadow Kiss
        (Vampire Academy, #3) Rating: nan . Ranking: 0.0
# 2946 The Shadows Rating: nan . Ranking: 0.0
# 2947 The Truth about the Devlins Rating: nan . Ranking: 0.0
# 2948 Ivanhoe Rating: nan . Ranking: 0.0
# 2949 The Master and Margarita R

In [13]:
#give a list sorted out with books you've already read:
# sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if (ratings_matrix[user_id, idx] > 0) or (np.isnan(pred_ratings_list[idx])):
        continue
    print("#", list_num , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

Top books are:
# 1 Harry Potter and the Sorcerer's Stone
        (Harry Potter, #1) Rating: 4.7
# 2 The da Vinci Code
        (Robert Langdon, #2) Rating: 4.3
# 3 Eclipse
        (The Twilight Saga, #3) Rating: 3.9
# 4 Breaking Dawn
        (The Twilight Saga, #4) Rating: 3.9
# 5 Pride and Prejudice Rating: 4.3
# 6 The Hobbit, or There and Back Again
        (The Lord of the Rings, #0) Rating: 4.5
# 7 The Help Rating: 4.6
# 8 The Lion, the Witch and the Wardrobe
        (Chronicles of Narnia, #1) Rating: 4.1
# 9 The Kite Runner Rating: 4.4
# 10 The Great Gatsby Rating: 3.5
# 11 The Lovely Bones Rating: 3.6
# 12 The Sea of Monsters
        (Percy Jackson and the Olympians, #2) Rating: 4.0
# 13 The Battle of the Labyrinth
        (Percy Jackson and the Olympians, #4) Rating: 3.9
# 14 A Game of Thrones
        (A Song of Ice and Fire, #1) Rating: 4.8
# 15 The Book Thief Rating: 4.6
# 16 The Diary of a Young Girl Rating: 4.6
# 17 The Fellowship of the Ring
        (The Lord of the Rings, #

# 191 We Should All Be Feminists Rating: 4.5
# 192 The Stinky Cheese Man and Other Fairly Stupid Tales Rating: 3.0
# 193 Aristotle and Dante Discover the Secrets of the Universe
        (Aristotle and Dante, #1) Rating: 4.5
# 194 Fourth Wing
        (The Empyrean, #1) Rating: 4.5
# 195 Persuasion Rating: 4.5
# 196 The Color Purple Rating: 3.0
# 197 The Tell-Tale Heart and Other Writings Rating: 4.5
# 198 Fifty Shades Freed
        (Fifty Shades, #3) Rating: 3.0
# 199 The Tale of Despereaux Rating: 4.5
# 200 Emma Rating: 3.0
# 201 The Queen of Nothing
        (The Folk of the Air, #3) Rating: 4.5
# 202 The Alchemyst
        (The Secrets of the Immortal Nicholas Flamel, #1) Rating: 4.5
# 203 The Other Boleyn Girl
        (The Plantagenet and Tudor Novels, #9) Rating: 4.5
# 204 Esperanza Rising Rating: 4.5
# 205 Chosen
        (House of Night, #3) Rating: 4.5
# 206 The Velveteen Rabbit Rating: 4.5
# 207 Legend
        (Legend, #1) Rating: 4.5
# 208 The BFG Rating: 4.5
# 209 Cress
        

        (Time Quintet, #3) Rating: 4.0
# 522 A Long Petal of the Sea Rating: 4.0
# 523 Shanghai Girls
        (Shanghai Girls, #1) Rating: 4.0
# 524 Her Fearful Symmetry Rating: 4.0
# 525 Cloudy With a Chance of Meatballs
        (Cloudy with a Chance of Meatballs, #1) Rating: 4.0
# 526 Prodigy
        (Legend, #2) Rating: 4.0
# 527 The Reader Rating: 4.0
# 528 Oliver Twist Rating: 4.0
# 529 Hex Hall
        (Hex Hall, #1) Rating: 4.0
# 530 Bleak House Rating: 4.0
# 531 The Color of Water: A Black Man's Tribute to His White Mother Rating: 4.0
# 532 Take My Hand Rating: 4.0
# 533 Orphan Train Rating: 4.0
# 534 The Ugly Truth
        (Diary of a Wimpy Kid, #5) Rating: 4.0
# 535 The Idea of You Rating: 4.0
# 536 The Bean Trees
        (Greer Family, #1) Rating: 4.0
# 537 The Tao of Pooh Rating: 4.0
# 538 Rodrick Rules
        (Diary of a Wimpy Kid, #2) Rating: 4.0
# 539 The Iron King
        (The Iron Fey, #1) Rating: 4.0
# 540 Transcendent Kingdom Rating: 4.0
# 541 Six of Crows
        (

# 692 Mansfield Park Rating: 4.0
# 693 Behind the Beautiful Forevers: Life, Death, and Hope in a Mumbai Undercity Rating: 4.0
# 694 The Last Straw
        (Diary of a Wimpy Kid, #3) Rating: 4.0
# 695 Northanger Abbey Rating: 4.0
# 696 Cold Mountain Rating: 4.0
# 697 Diary of a Wimpy Kid
        (Diary of a Wimpy Kid, #1) Rating: 4.0
# 698 Hush, Hush
        (Hush, Hush, #1) Rating: 4.0
# 699 We Were Liars Rating: 4.0
# 700 The Island of Dr. Moreau Rating: 4.0
# 701 4th of July
        (Women's Murder Club, #4) Rating: 4.0
# 702 Iron Flame
        (The Empyrean, #2) Rating: 4.0
# 703 Dead Poets Society Rating: 3.0
# 704 Robinson Crusoe Rating: 3.0
# 705 Jonathan Livingston Seagull Rating: 3.0
# 706 Storm Front
        (The Dresden Files, #1) Rating: 3.0
# 707 The Diving Bell and the Butterfly: A Memoir of Life in Death Rating: 3.0
# 708 Waiting for Godot Rating: 3.0
# 709 The Twits Rating: 3.0
# 710 The Crown
        (The Selection, #5) Rating: 3.0
# 711 The Phantom of the Opera Rating:

In [14]:
idx = 8
this_ratings = ratings_matrix[indices[0,idx]]
print(this_ratings)

my_ratings = ratings_matrix[indices[0,0]]
print(my_ratings)

for i, rating in enumerate(this_ratings):
    if rating > 0 and my_ratings[i]>0:
        print("-", titles[i], ", their Rating:", rating, " My Rating:", my_ratings[i])

# print(distances, indices[-1,-1])

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
- The Fiery Cross
        (Outlander, #5) , their Rating: 5.0  My Rating: 4.0
- The Fault in Our Stars , their Rating: 5.0  My Rating: 4.0
- To Kill a Mockingbird , their Rating: 5.0  My Rating: 5.0
- Drums of Autumn
        (Outlander, #4) , their Rating: 5.0  My Rating: 5.0
- Holes
        (Holes, #1) , their Rating: 5.0  My Rating: 4.0
- The Hunger Games
        (The Hunger Games, #1) , their Rating: 5.0  My Rating: 5.0
- Mockingjay
        (The Hunger Games, #3) , their Rating: 5.0  My Rating: 5.0
- Dragonfly in Amber
        (Outlander, #2) , their Rating: 5.0  My Rating: 5.0
- Outlander
        (Outlander, #1) , their Rating: 5.0  My Rating: 5.0
- Twilight
        (The Twilight Saga, #1) , their Rating: 4.0  My Rating: 3.0


In [15]:
#find most similar books using cosine similarity
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(ratings_matrix.T)

similarity_df = pd.DataFrame(similarity_matrix, index=titles, columns=titles)

# Function to get k nearest neighbors for a movie
def get_similar_book(book_name, k=3):
    similar_book = similarity_df[book_name].sort_values(ascending=False)[1:k+1]
    return similar_book

book_name = 'First Lie Wins'
print("\nTop 5 similar book to", book_name, ":")
print(get_similar_book(book_name, k=5))


Top 5 similar book to First Lie Wins :
Listen for the Lie             0.571406
She's Not Sorry                0.555256
Darling Girls                  0.543448
The Last One at the Wedding    0.517413
The Fury                       0.516598
Name: First Lie Wins, dtype: float64


In [16]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

# Example user rating data (rows = users, columns = items)
ratings_df = pd.DataFrame(ratings)

# Step 1: Handle missing values using imputation (we will fill missing values with the mean rating)
imputer = SimpleImputer(strategy='mean')
ratings_filled = imputer.fit_transform(ratings_df)

# Step 2: Apply KMeans clustering
# We will use 2 clusters as an example
kmeans = KMeans(n_clusters=int(num_users/10), random_state=42)
clusters = kmeans.fit_predict(ratings_filled)

# Step 3: Add the cluster labels to the original DataFrame
ratings_df['Cluster'] = clusters

# # Print the user ratings with cluster assignments
# print("\nUser Ratings with Clusters:")
# print(ratings_df)

# # Step 4: Print the cluster centers (the centroid of each cluster)
# print("\nCluster Centers (Centroids):")
# print(kmeans.cluster_centers_)


user_id = 0
# print(clusters)

cluster_this_user = clusters[user_id]
# print(cluster_this_user)

pred_ratings_list = kmeans.cluster_centers_[user_id]
# print(pred_ratings_list)
# for i in range(len())

#give a list sorted out with books you've already read:
sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if (ratings_matrix[user_id, idx] > 0) or (np.isnan(pred_ratings_list[idx])) :
        continue
    print("#", list_num , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Top books are:
# 1 Pride and Prejudice Rating: 3.6
# 2 The Adventures of Huckleberry Finn Rating: 3.3
# 3 Little Women Rating: 3.2
# 4 The Diary of a Young Girl Rating: 2.9
# 5 Lord of the Flies Rating: 2.6
# 6 The Hobbit, or There and Back Again
        (The Lord of the Rings, #0) Rating: 2.5
# 7 Where the Sidewalk Ends Rating: 2.5
# 8 The Great Gatsby Rating: 2.5
# 9 The Fellowship of the Ring
        (The Lord of the Rings, #1) Rating: 2.3
# 10 Of Mice and Men Rating: 2.3
# 11 The Odyssey Rating: 2.2
# 12 Harry Potter and the Sorcerer's Stone
        (Harry Potter, #1) Rating: 2.1
# 13 Sense and Sensibility Rating: 1.8
# 14 Memoirs of a Geisha Rating: 1.8
# 15 1984 Rating: 1.7
# 16 The da Vinci Code
        (Robert Langdon, #2) Rating: 1.4
# 17 Moby-Dick or, The Whale Rating: 1.4
# 18 The Count of Monte Cristo Rating: 1.2
# 19 Gone with the Wind Rating: 1.1
# 20 Frankenstein Rating: 1.1
# 21 The Princess Bride Rating: 1.0
# 22 Brave New World Rating: 0.9
# 23 Jurassic Park
        (

# 238 Ivanhoe Rating: 0.1
# 239 Mr. Maybe Rating: 0.1
# 240 Peter Pan Rating: 0.1
# 241 Inferno Rating: 0.1
# 242 Shoe Dog: A Memoir by the Creator of Nike Rating: 0.1
# 243 The Curious Incident of the Dog in the Night-Time Rating: 0.1
# 244 The Legend of Sleepy Hollow Rating: 0.1
# 245 The Swiss Family Robinson Rating: 0.1
# 246 Through the Looking-Glass and What Alice Found There
        (Alice's Adventures in Wonderland, #2) Rating: 0.1
# 247 The Tale of Peter Rabbit
        (World of Beatrix Potter, #1) Rating: 0.1
# 248 Old Yeller Rating: 0.1
# 249 The Awakening Rating: 0.1
# 250 The Name of the Rose Rating: 0.1
# 251 Roll of Thunder, Hear My Cry
        (Logans, #4) Rating: 0.1
# 252 A Little Princess Rating: 0.1
# 253 The Lowland Rating: 0.1
# 254 Water for Elephants Rating: 0.1
# 255 The Metamorphosis Rating: 0.1
# 256 Can You Keep a Secret? Rating: 0.1
# 257 Redeeming Love Rating: 0.1
# 258 Julius Caesar Rating: 0.1
# 259 Foundation
        (Foundation, #1) Rating: 0.1
# 260 P

# 503 The Wife Between Us Rating: 0.0
# 504 Ordinary Grace Rating: 0.0
# 505 Love You Forever Rating: 0.0
# 506 Untamed Rating: 0.0
# 507 Is Everyone Hanging Out Without Me? Rating: 0.0
# 508 Fight Club Rating: 0.0
# 509 Sheâ€™s Come Undone Rating: 0.0
# 510 Where'd You Go, Bernadette Rating: 0.0
# 511 Breakfast of Champions Rating: 0.0
# 512 The Inheritance Games
        (The Inheritance Games, #1) Rating: 0.0
# 513 Red, White & Royal Blue Rating: 0.0
# 514 The Last Flight Rating: 0.0
# 515 November 9 Rating: 0.0
# 516 Cloud Atlas Rating: 0.0
# 517 Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values
        (Phaedrus, #1) Rating: 0.0
# 518 One of Us Is Lying
        (One of Us is Lying, #1) Rating: 0.0
# 519 The Dutch House Rating: 0.0
# 520 The Runaway Jury Rating: 0.0
# 521 The 7 Habits of Highly Effective People: Powerful Lessons in Personal Change Rating: 0.0
# 522 Little House in the Big Woods
        (Little House, #1) Rating: 0.0
# 523 Needful Things Rating: 0.0
#

# 761 The Exchange
        (The Firm, #2) Rating: 0.0
# 762 Bleachers Rating: 0.0
# 763 Echo Park
        (Harry Bosch, #12; Harry Bosch Universe, #17) Rating: 0.0
# 764 90 Minutes in Heaven: A True Story of Death and Life Rating: 0.0
# 765 Daemon
        (Daemon, #1) Rating: 0.0
# 766 The Poet X Rating: 0.0
# 767 Detransition, Baby Rating: 0.0
# 768 The Gunslinger Born
        (Stephen King's The Dark Tower: Beginnings, #1) Rating: 0.0
# 769 Mostly Harmless
        (Hitchhiker's Guide to the Galaxy, #5) Rating: 0.0
# 770 An Unquiet Mind: A Memoir of Moods and Madness Rating: 0.0
# 771 The Age of Miracles Rating: 0.0
# 772 Lady Tan’s Circle of Women Rating: 0.0
# 773 Cursor's Fury
        (Codex Alera, #3) Rating: 0.0
# 774 The Jane Austen Book Club Rating: 0.0
# 775 The Darkest Minds
        (The Darkest Minds, #1) Rating: 0.0
# 776 First Comes Love Rating: 0.0
# 777 Final Offer
        (Dreamland Billionaires, #3) Rating: 0.0
# 778 Every Heart a Doorway
        (Wayward Children, #1)

...

# 1379 Girl in Translation Rating: 0.0
# 1380 The Trial Rating: 0.0
# 1381 The Remains of the Day Rating: 0.0
# 1382 A Light in the Attic Rating: 0.0
# 1383 Talking to Strangers: What We Should Know About the People We Don't Know Rating: 0.0
# 1384 One Thousand White Women: The Journals of May Dodd
        (One Thousand White Women, #1) Rating: 0.0
# 1385 Summer of '69 Rating: 0.0
# 1386 The Hating Game Rating: 0.0
# 1387 Heidi Rating: 0.0
# 1388 The Bonesetter's Daughter Rating: 0.0
# 1389 Jaws
        (Jaws, #1) Rating: 0.0
# 1390 The Inmate Rating: 0.0
# 1391 Darkfever
        (Fever, #1) Rating: 0.0
# 1392 The Passage
        (The Passage, #1) Rating: 0.0
# 1393 The Bookish Life of Nina Hill Rating: 0.0
# 1394 The Hero of Ages
        (Mistborn, #3) Rating: 0.0
# 1395 The End of Her Rating: 0.0
# 1396 Snow Rating: 0.0
# 1397 A Long Way Down Rating: 0.0
# 1398 The Prince Rating: 0.0
# 1399 All Good People Here Rating: 0.0
# 1400 Grip of the Shadow Plague
        (Fablehaven, #3) Rat

        (Mistborn, #2) Rating: 0.0
# 1690 Girl, Interrupted Rating: 0.0
# 1691 Foundation and Empire
        (Foundation, #2) Rating: 0.0
# 1692 That Summer Rating: 0.0
# 1693 Transcendent Kingdom Rating: 0.0
# 1694 The Last Bookshop in London Rating: 0.0
# 1695 In a Dark, Dark Wood Rating: 0.0
# 1696 Hounded
        (The Iron Druid Chronicles, #1) Rating: 0.0
# 1697 Steppenwolf Rating: 0.0
# 1698 Rilla of Ingleside
        (Anne of Green Gables, #8) Rating: 0.0
# 1699 Behold the Dreamers Rating: 0.0
# 1700 The Mark of Athena
        (The Heroes of Olympus, #3) Rating: 0.0
# 1701 The Pilot's Wife
        (Fortune's Rocks Quartet, #3) Rating: 0.0
# 1702 Part of Your World
        (Part of Your World, #1) Rating: 0.0
# 1703 Silver Girl Rating: 0.0
# 1704 Iron Flame
        (The Empyrean, #2) Rating: 0.0
# 1705 This Is How You Lose Her Rating: 0.0
# 1706 Ward D Rating: 0.0
# 1707 The Lost Bookshop Rating: 0.0
# 1708 The Historian Rating: 0.0
# 1709 Entwined with You
        (Crossfire, #3

# 2006 A Flicker in the Dark Rating: 0.0
# 2007 Eats, Shoots & Leaves: The Zero Tolerance Approach to Punctuation Rating: 0.0
# 2008 Twelve Years a Slave Rating: 0.0
# 2009 I'll Give You the Sun Rating: 0.0
# 2010 The Magicians
        (The Magicians, #1) Rating: 0.0
# 2011 My Horizontal Life: A Collection of One-Night Stands Rating: 0.0
# 2012 Eye of the Needle Rating: 0.0
# 2013 The Queen's Fool
        (The Plantagenet and Tudor Novels, #12) Rating: 0.0
# 2014 End of Story Rating: 0.0
# 2015 How to Win Friends & Influence People Rating: 0.0
# 2016 The Diviners
        (The Diviners, #1) Rating: 0.0
# 2017 The Mother-in-Law Rating: 0.0
# 2018 1Q84
        (1Q84, #1-3) Rating: 0.0
# 2019 White Teeth Rating: 0.0
# 2020 Last Sacrifice
        (Vampire Academy, #6) Rating: 0.0
# 2021 Mayflower: A Story of Courage, Community, and War Rating: 0.0
# 2022 The Omnivore's Dilemma: A Natural History of Four Meals Rating: 0.0
# 2023 The Exorcist Rating: 0.0
# 2024 The Bone Collector
        (Lin

# 2331 The Plot Against America Rating: -0.0
# 2332 Circling the Sun Rating: -0.0
# 2333 It's Kind of a Funny Story Rating: -0.0
# 2334 The Year of the Flood
        (MaddAddam, #2) Rating: -0.0
# 2335 American Born Chinese Rating: -0.0
# 2336 Hearts in Atlantis Rating: -0.0
# 2337 The House at Riverton Rating: -0.0
# 2338 All Fall Down Rating: -0.0
# 2339 The Lacuna Rating: -0.0
# 2340 Schindlerâ€™s List Rating: -0.0
# 2341 Son
        (The Giver, #4) Rating: -0.0
# 2342 All the Lies
        (Mindf*ck, #4) Rating: -0.0
# 2343 Perfect
        (Pretty Little Liars, #3) Rating: -0.0
# 2344 The Elfstones of Shannara
        (The Original Shannara Trilogy, #2) Rating: -0.0
# 2345 Blood Song
        (Raven's Shadow, #1) Rating: -0.0
# 2346 Long Island
        (Eilis Lacey, #2) Rating: -0.0
# 2347 Dead Ever After
        (Sookie Stackhouse, #13) Rating: -0.0
# 2348 Christmas Presents Rating: -0.0
# 2349 Pinocchio Rating: -0.0
# 2350 At the Water's Edge Rating: -0.0
# 2351 Those Who Leave and

# 2656 Joyland Rating: -0.0
# 2657 The Fury Rating: -0.0
# 2658 Dreamcatcher Rating: -0.0
# 2659 Halfway to the Grave
        (Night Huntress, #1) Rating: -0.0
# 2660 The Purpose Driven Life: What on Earth Am I Here for? Rating: -0.0
# 2661 The Complete Maus Rating: -0.0
# 2662 Tuck Everlasting Rating: -0.0
# 2663 Recursion Rating: -0.0
# 2664 Good Bad Girl Rating: -0.0
# 2665 The Wedding
        (The Notebook, #2) Rating: -0.0
# 2666 Just Mercy Rating: -0.0
# 2667 The Slow Regard of Silent Things
        (The Kingkiller Chronicle, #2.5) Rating: -0.0
# 2668 Reamde Rating: -0.0
# 2669 The Wide Window
        (A Series of Unfortunate Events, #3) Rating: -0.0
# 2670 Frostbite
        (Vampire Academy, #2) Rating: -0.0
# 2671 Five Quarters of the Orange Rating: -0.0
# 2672 All the King's Men Rating: -0.0
# 2673 Tara Road Rating: -0.0
# 2674 Beach Music Rating: -0.0
# 2675 The Boxcar Children
        (The Boxcar Children, #1) Rating: -0.0
# 2676 Maisie Dobbs
        (Maisie Dobbs, #1) Ratin

In [ ]:
#doing masked autoencoder
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset, random_split

# Mask for observed values (1 for observed, 0 for missing)
ratings_torch = torch.tensor(ratings).float()
mask = (ratings_torch != 0).float()
mask_tensor = torch.tensor(mask)


#Define autoencoder
class SparseAutoencoder(nn.Module):
    def __init__(self, num_items, latent_dim):
        super(SparseAutoencoder, self).__init__()
        hidden1 = latent_dim*2
        self.encoder1 = nn.Linear(num_items, hidden1)
        self.encoder2 = nn.Linear(hidden1, latent_dim)
        self.decoder1 = nn.Linear(latent_dim, hidden1)
        self.decoder2 = nn.Linear(hidden1, num_items)
        
    def forward(self, x):
        x = torch.relu(self.encoder1(x))
        x = torch.relu(self.encoder2(x))
        x = torch.relu(self.decoder1(x))
        x = self.decoder2(x)
        # Scale sigmoid output to [1, 5]
        return 1 + 4 * torch.sigmoid(x)

    
#initialize the model
num_users, num_items = ratings_torch.shape
np.save("num_users.npy", np.array(num_users))
np.save("num_items.npy", np.array(num_items))
latent_dim = int(num_items/8) # Number of latent features

model = SparseAutoencoder(num_items, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Use MSE loss but only consider observed values
def masked_mse_loss(reconstructed, original, mask):
    loss = ((reconstructed - original) ** 2) * mask
    return loss.sum() / mask.sum()

#break up data into train and val
dataset = TensorDataset(ratings_torch, mask_tensor) #keeping the mask
print("ratings_torch shape =", ratings_torch.shape)
print(len(dataset))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("len(train_loader) = ", len(train_loader))
print("len(val_loader) = ", len(val_loader))

#train the model
epochs = 5000
best_loss = 10e10
counter = 0
for epoch in range(epochs):
    train_loss = 0.0
    for inputs, mask in train_loader:
        model.train()
        optimizer.zero_grad()

        # Forward pass
        reconstructed = model(inputs)
        loss = masked_mse_loss(reconstructed, inputs, mask)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    train_loss /= len(train_loader)
    
    if (epoch + 1) % 10 == 0:
#         print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss}")
        #check validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, mask in val_loader:
                outputs = model(inputs)
                loss = masked_mse_loss(outputs, inputs, mask)
                val_loss += loss.item()
                

        val_loss /= len(val_loader)
        

        print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")
        if val_loss < best_loss: #if improve then save
            #save model 
            torch.save(model.state_dict(), "2model{}.pkl".format(latent_dim))
            best_loss = val_loss
            print("Model saved to model{}.pkl.".format(latent_dim))
            counter = 0
        else:
            counter += 10
            
    if counter > 200:
        print("Done training because of no improvement.")
        break
        
            


/tmp/ipykernel_2614/1224755660.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_tensor = torch.tensor(mask)


ratings_torch shape = torch.Size([1687, 3023])
1687
len(train_loader) =  43
len(val_loader) =  11
Epoch 10 - Train Loss: 0.9283 - Val Loss: 0.9306
Model saved to model377.pkl.
Epoch 20 - Train Loss: 0.9347 - Val Loss: 0.9514
Epoch 30 - Train Loss: 0.9469 - Val Loss: 0.9653
Epoch 40 - Train Loss: 0.9186 - Val Loss: 0.9511
Epoch 50 - Train Loss: 0.9110 - Val Loss: 0.9425
Epoch 60 - Train Loss: 0.9064 - Val Loss: 0.9424
Epoch 70 - Train Loss: 0.8999 - Val Loss: 0.9467
Epoch 80 - Train Loss: 0.9049 - Val Loss: 0.9406
Epoch 90 - Train Loss: 0.9251 - Val Loss: 0.9380
Epoch 100 - Train Loss: 0.9088 - Val Loss: 0.9390
Epoch 110 - Train Loss: 0.8974 - Val Loss: 0.9365
Epoch 120 - Train Loss: 0.9232 - Val Loss: 0.9357
Epoch 130 - Train Loss: 0.8952 - Val Loss: 0.9338


In [ ]:
num_users, num_items

In [ ]:
print(sum(sum(mask)))
print(mask.shape)
print(mask.shape[0] * mask.shape[1])
print(sum(sum(mask)) / (mask.shape[0] * mask.shape[1]))



In [ ]:
#Evaulating the model
model.eval()
with torch.no_grad():
    reconstructed = model(ratings_torch)

# Fill missing values in the original matrix
filled_data = ratings_torch.clone()
filled_data[mask_tensor == 0] = reconstructed[mask_tensor == 0]

print("Original Data:\n", ratings_torch)
print("Reconstructed Data:\n", reconstructed)
print("Filled Data:\n", filled_data)


In [ ]:
print(reconstructed[0].numpy())
print(ratings_torch[0].numpy())

print((reconstructed[0].numpy()-ratings_torch[0].numpy())/ratings_torch[0].numpy())

In [ ]:
pred_ratings_list = reconstructed[0].detach().numpy()

#give a list sorted out with books you've already read:
sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if  (ratings_matrix[user_id, idx] > 0) or(np.isnan(pred_ratings_list[idx])) :
        continue
    print("#", list_num , titles[idx], " - Predicted Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

In [ ]:
#making weighted loss matrix
percents = np.array([ 2.0839861,   6.38564535, 22.8939068,  37.94135873, 30.69510302])
each_weights = 100/percents
print(each_weights)
print(each_weights.sum())

print(each_weights * percents)

weights_array = np.zeros(ratings_torch.shape)
for i in tqdm(range(len(ratings_torch))):
    for j in range(len(ratings_torch[0])):
        for num in [1, 2, 3, 4, 5]:
            if ratings_torch[i,j] == num:
                weights_array[i,j] = each_weights[num-1]
weights_tensor = torch.tensor(weights_array)

In [ ]:
weights[0]

In [ ]:
#doing masked autoencoder with weighted loss
latent_dim = 100 # Number of latent features

model = SparseAutoencoder(num_items, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Use MSE loss with weights but only consider observed values
def masked_mse_loss_diff(reconstructed, original, mask, weights):
    loss = (((reconstructed - original) ** 2) * mask)
    weighted_loss = loss * weights
    return weighted_loss.sum() / mask.sum()/100

#break up data into train and val
print("ratings_torch shape = ", ratings_torch.shape)
print("mask_tensor shape = ", mask_tensor.shape)
print("weights shape = ", weights_tensor.shape)

dataset = TensorDataset(ratings_torch, mask_tensor, weights_tensor) #keeping the mask
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


#train the model
epochs = 5000
best_loss = 10e10
counter = 0
for epoch in range(epochs):
    train_loss = 0.0
    for inputs, mask, this_weight in train_loader:
        model.train()
        optimizer.zero_grad()

        # Forward pass
        reconstructed = model(inputs)
        loss = masked_mse_loss_diff(reconstructed, inputs, mask, this_weight)
#         loss_not_weighted = masked_mse_loss(reconstructed, inputs, mask)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    train_loss /= len(train_loader)
    
    if (epoch + 1) % 10 == 0:
#         print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss}")
        #check validation
        model.eval()
        val_loss = 0.0
        val_loss_not_weighted = 0.0
        with torch.no_grad():
            for inputs, mask, this_weight in val_loader:
                outputs = model(inputs)
                loss = masked_mse_loss_diff(outputs, inputs, mask, this_weight)
                loss_not_weighted = masked_mse_loss(outputs, inputs, mask)
                val_loss += loss.item()
                val_loss_not_weighted += loss_not_weighted.item()
                

        val_loss /= len(val_loader)
        

        print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val not weighted: {val_loss_not_weighted:.4f}" )
        if val_loss < best_loss: #if improve then save
            #save model 
            torch.save(model.state_dict(), "model_weighted{}.pkl".format(latent_dim))
            best_loss = val_loss
            print("Model saved to model_weighted{}.pkl.".format(latent_dim))
            counter = 0
        else:
            counter += 10
            
    if counter > 200:
        print("Done training because of no improvement.")
        break
        
            


In [ ]:
dfghj
import torch
from sklearn.model_selection import KFold

# Mask for observed values (1 for observed, 0 for missing)
ratings_torch = torch.tensor(ratings).float()
mask = (ratings_torch != 0).float()
print(mask)


#Define autoencoder
class SparseAutoencoder(nn.Module):
    def __init__(self, num_items, latent_dim):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Linear(num_items, latent_dim)
        self.decoder = nn.Linear(latent_dim, num_items)
        
    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        # Scale sigmoid output to [1, 5]
        return 1 + 4 * torch.sigmoid(decoded)
        return decoded

    
#initialize the model
num_users, num_items = ratings_torch.shape
np.save("num_users.npy", np.array(num_users))
np.save("num_items.npy", np.array(num_items))

for latent_dim in [2, 5, 10, 20, 40, 50, 75, 100]:
    print("latent_dim = ", latent_dim)
# latent_dim = 20  # Number of latent features

    model = SparseAutoencoder(num_items, latent_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Define your model, loss function, and optimizer
    # Assuming model, ratings_torch, mask, and optimizer are already defined

    epochs = 1000
    k_folds = 5  # Number of folds for cross-validation
    kf = KFold(n_splits=k_folds, shuffle=True)

    # Store the losses for each fold
    fold_losses = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(ratings_torch)):
#         print(f"\nFold {fold + 1}/{k_folds}")

        # Split the data into training and validation sets
        train_ratings = ratings_torch[train_idx]
        val_ratings = ratings_torch[val_idx]
        train_mask = mask[train_idx]
        val_mask = mask[val_idx]

        # Initialize a new model for each fold
        model = SparseAutoencoder(num_items, latent_dim)
    #     optimizer = optim.Adam(model.parameters(), lr=0.01)

        # Re-initialize optimizer for each fold
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        # Train the model
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()

            # Forward pass for training
            reconstructed = model(train_ratings)
            loss = masked_mse_loss(reconstructed, train_ratings, train_mask)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

#             if (epoch + 1) % 1000 == 0:
#                 print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

        # Evaluate the model on the validation set
        model.eval()
        with torch.no_grad():
            reconstructed_val = model(val_ratings)
            val_loss = masked_mse_loss(reconstructed_val, val_ratings, val_mask)

        print(f"Validation Loss for Fold {fold + 1}: {val_loss.item():.4f}")

        # Store the validation loss for this fold
        fold_losses.append(val_loss.item())

    # Print the average validation loss after all folds
    print(f"\nAverage Validation Loss across all folds: {sum(fold_losses)/k_folds:.4f}")


In [ ]:
fgh

In [ ]:
ratings_matrix[user_id]

In [ ]:
# Convert the filled ratings data into a numpy array
ratings_matrix = ratings_df.values

# Initialize KNN (using user-based KNN)
import math
knn = NearestNeighbors(n_neighbors=math.ceil(num_users/10), metric='cosine')  # Using cosine similarity
knn.fit(ratings_matrix)

# Example: Predict rating for User 0 on Item 2
user_id = 0  # Index of user in the matrix
item_id = 9  # Index of item to predict rating for

# Get the nearest neighbors for user 0 (excluding the user itself)
distances, indices = knn.kneighbors([ratings_matrix[user_id]])

pred_ratings_list = np.array([])
rankings_list = np.array([])
for item_id in range(num_titles):
    # Get the ratings for the neighbors on item 2
    neighbor_ratings = np.array([ratings_matrix[i, item_id] for i in indices[0] if not np.isnan(ratings_matrix[i, item_id])])
    print(ratings_matrix[:, item_id])
    print(neighbor_ratings)
    ghjk
    
    predicted_rating = np.mean(neighbor_ratings[np.nonzero(neighbor_ratings)])
    rankings = np.sum(neighbor_ratings[np.nonzero(neighbor_ratings)])

#     print(predicted_rating)

    pred_ratings_list = np.append(pred_ratings_list, predicted_rating)
    rankings_list = np.append(rankings_list, rankings)
#     print(f"Predicted rating for User {user_id} on Item {item_id}: {predicted_rating}")
    
best_book_rating = np.max(pred_ratings_list)
best_book_idx = np.argmax(pred_ratings_list)

sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
for i, idx in enumerate(sorted_indices): 
    print("#", (i+1) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))

In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds

# Create a sparse matrix (CSR format)
A = np.array([[1.0, 0, 0], [0, 2, 3], [4, 0, 6], [0, 0, 0]])
# print(A)
sparse_matrix = sp.csr_matrix(ratings_matrix)

# Perform SVD on the sparse matrix
# k is the number of singular values to compute (you can choose a value smaller than min(m, n))
U, S, VT = svds(sparse_matrix, k=500)

# Output the matrices
print("U (Left Singular Vectors):\n", U)
print("\nS (Singular Values):\n", S)
print("\nVT (Right Singular Vectors - Transposed):\n", VT)

# Reconstruct the matrix from U, S, VT
S_full = np.diag(S)  # Convert singular values to a diagonal matrix
A_reconstructed = np.dot(U, np.dot(S_full, VT))

print("\nReconstructed Matrix A:\n", A_reconstructed)


In [ ]:
my_diff = (ratings_matrix[0]- A_reconstructed[0])
print(ratings_matrix.shape)
plt.plot(my_diff, '.')

# for i in range(len(ratings_matrix[0])):
#     if ratings_matrix[0, i] > 0:
#         print(ratings_matrix[0, i], A_reconstructed[0, i], titles[i])
        
for i in range(len(ratings_matrix[0])):
    if ratings_matrix[0, i] == 0 and A_reconstructed[0, i] > 0:
        print(ratings_matrix[0, i], A_reconstructed[0, i], titles[i])


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Create a custom colormap with white for 0 and red for non-zero
cmap = mcolors.ListedColormap(['white', 'red'])
bounds = [0, 0.1, 1]  # Set bounds for 0 (white) and non-zero (red)
norm = mcolors.BoundaryNorm(bounds, cmap.N)

# plt.imshow(ratings_matrix - A_reconstructed, cmap=cmap, norm=norm)
plt.plot(ratings_matrix - A_reconstructed)
plt.show()

In [ ]:
ratings_matrix - A_reconstructed

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.75, min_samples=2, metric='cosine')
labels = dbscan.fit_predict(ratings_matrix)


In [ ]:
print(list(set(labels)))
print(labels.shape)

In [ ]:
idx_in_group = np.arange(len(labels))
filtered_users = [idx_in_group for idx_in_group, label in zip(idx_in_group, labels) if label == 0]
print(filtered_users)

In [ ]:
from sklearn.cluster import SpectralClustering
from scipy.sparse import csr_matrix
import numpy as np

# Example sparse data
# X = np.random.rand(100, 2)
X_sparse = csr_matrix(ratings_matrix)

n_clusters = 50
# Apply Spectral Clustering
spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors')
labels = spectral.fit_predict(X_sparse)

print(labels)
print(list(set(labels)))
print(labels.shape)

In [ ]:
# idx_in_group = np.arange(len(labels))
# filtered_users = [idx_in_group for idx_in_group, label in zip(idx_in_group, labels) if label == 0]
# print(filtered_users)

group_averages = []

for group in range(n_clusters):
    # Find indices of users in the current group
    group_users = np.where(labels == group)[0]
    
    # Extract the rows for users in this group
    group_data = ratings_matrix[group_users]
    
    print("Number of perople in group = ", group_data.shape[0])
    
    pred_ratings_list = np.array([])
    for item_id in range(num_titles):
        data = group_data[:,item_id]
        predicted_rating = np.mean(data[np.nonzero(data)])
#                 rankings = np.sum(neighbor_ratings[np.nonzero(neighbor_ratings)])

        pred_ratings_list = np.append(pred_ratings_list, predicted_rating)
    # Compute the average for each column (item) for this group
#     group_avg = np.mean(group_data[np.nonzero(group_data)], axis=0)
    
    # Append the average for this group
    group_averages.append(pred_ratings_list)

# Convert the list of group averages to a numpy array for easy viewing
group_averages = np.array(group_averages)

# Display the average for each item in each group
print("Average preferences for each item by group:")
print(group_averages.shape)

In [ ]:
group = labels[0]
print("my group = ", group)
sorted_indices = np.argsort(group_averages[group])[::-1]
print(sorted_indices)
for i in sorted_indices:
    if (ratings_matrix[0, i] > 0) or (np.isnan(group_averages[group, i])):
        pass
    else:
        print(titles[i], round(group_averages[group,i], 1))